# This code was used to iterate over and download IFRC Evaluations.
It may or may not run as-is... I haven't cleaned it up yet but wanted to check it in ASAP as per discussion with Dan.

In [1]:
import requests

from bs4 import BeautifulSoup
import pandas as pd

import sys
import os
import re

In [2]:
base_url = 'https://www.ifrc.org/evaluations'
corpora_location = '..\\corpora\\ifrc_evaluations'


In [3]:
def get_mainpage_content(soup):
    

   # r = requests.get(base_url)
   # soup = BeautifulSoup(r.text, 'html.parser')
    results = []
    
    table = soup.find("tbody")
    rows = table.find_all("tr")
    for row in rows:
        cols = row.find_all("td")

        for col in cols:
            if col['headers'][0] == 'view-evaluation-name-table-column':
                media_url = 'https://www.ifrc.org' + col.contents[0].attrs['href']
                #print(media_url)
        cols = [col.text.strip() for col in cols]
        cols.append(media_url)
        results.append(cols)
        
    #go to next page
    next_page = ''
    next_url = 'end'
    for x in soup.findAll('a'):
        #print(x.attrs)
        if(x.get('title') == 'Go to next page'):
            next_page = x['href']

    if len(next_page) > 1:
        next_url = base_url + next_page
        
    return results, next_url
    

In [4]:
def prep_mainpage(base_url):
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, 'html.parser')    
    
    header = soup.find("thead")
    rows = header.find_all("tr")
    for row in rows:
        cols = row.find_all("th")


        headers = [col.text.strip() for col in cols]
        headers.append('media_url')
        
        
    #go to next page
    next_page = ''
    next_url = 'end'
    for x in soup.findAll('a'):
        #print(x.attrs)
        if(x.get('title') == 'Go to next page'):
            next_page = x['href']

    if len(next_page) > 1:
        next_url = base_url + next_page

    
    return headers, next_url

In [5]:
all_results = []
headers, next_url = prep_mainpage(base_url)

while next_url != 'end':
    print(next_url)
    r = requests.get(next_url)
    soup = BeautifulSoup(r.text, 'html.parser')    
    
    results, next_url = get_mainpage_content(soup)
    for row in results:
        all_results.append(row)
    print(len(all_results))

https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=1
20
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=2
40
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=3
60
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=4
80
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=5
100
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=6
120
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=7
140
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=8
160
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=9
180
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=10
200
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=11
220
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=12
240
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=13
260
https://www.ifrc.org/evaluations?date_from=&date_to=&text=&page=14
280
https://www.ifrc.or

In [6]:
df = pd.DataFrame(all_results, columns=headers)
df = df.drop(columns =["Management response"])


In [7]:
def get_link_to_downloadable_asset(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    results = soup.find_all('a', {"class": "col-3-md download-links__download-link"})
    if len(results) != 1:
        return f"error - {len(results)} possible results found"
    
    return results[0].attrs['href']

#Get the url to the page where the download link can be found
df['document_url'] = df['media_url'].apply(get_link_to_downloadable_asset)


In [8]:
#some more df cleanup
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['year'] = df['Date'].dt.year
df['doc_extension'] = df['document_url'].apply(lambda x: x[-3:])
df['doc_filename'] = df['year'].astype(str) + '_' + df['Title'].apply(lambda x: re.sub(r'\W+', '_', x).lower()) + '.' + df['doc_extension']
df['doc_filename'] = df['doc_filename'].apply(lambda x: x.encode('ASCII', 'ignore').decode('utf-8'))

In [26]:
def download_docs(url, filename):

    #url = "https://adore.ifrc.org/Download.aspx?FileId=179783&.pdf"
    #filename = filename
    filepath = os.path.join(os.path.relpath(corpora_location),filename)

    response = requests.get(url)

    if response.status_code == 200:
        with open(filepath, "wb") as f:
            f.write(response.content)
            print(f"File {filename} saved to disk.")
    else:
        print(f"Download failed: HTTP status code {response.status_code}")
        

    return response.status_code

#download the docs
df2['download_status'] = df2.apply(lambda x: download_docs(x['document_url'], x['doc_filename']), axis=1)
#df['sum'] = df.apply(lambda row: add(row['column1'], row['column2']), axis=1)



File 2009_review_of_the_international_federation_s_shelter_cluster_commitment.pdf saved to disk.
File 2009_inter_agency_real_time_evaluation_ia_rte_of_the.pdf saved to disk.
File 2009_end_term_evaluation_of_red_cross_and_red_crescent.pdf saved to disk.
File 2009_review_mission_to_southern_africa_zone_and_mozambique_25_29_may_2009.doc saved to disk.
File 2009_iasc_cluster_approach_evaluation_phase_2_myanmar_country_study.zip saved to disk.
File 2009_campaign_leadership_survey_report_2009.pdf saved to disk.
File 2009_review_of_the_fiji_red_cross_hiv_aids_and_blood_iec_materials.pdf saved to disk.
File 2009_review_of_the_nrcs_icrc_micro_economic_intiative_mei_programme.zip saved to disk.
File 2009_review_mission_to_panama_zone_and_peru_2_10_december_2009.doc saved to disk.
File 2009_cluster_approach_evaluation_2_synthesis_report.zip saved to disk.
File 2009_evaluation_of_the_red_cross_response_to_typhoons_ketsana_and_mirinae_disasters_in_viet_nam.pdf saved to disk.
File 2009_evaluation_of

File 2007_evaluation_of_specific_rehabilitation_and_reconstruction_projects_special_tsunami_plan_pem_framework.pdf saved to disk.
File 2007_maldives_community_sewerage_projectevaluation.pdf saved to disk.
File 2007_health_information_from_disease_surveillance_mds_sri_lanka.pdf saved to disk.
File 2007_preliminary_impact_evaluation_of_the_transitional_shelter_programme_in_aceh_province_indonesia.pdf saved to disk.
File 2007_evaluation_report_supplementary_water_supply_system_ifrc_maldives.pdf saved to disk.
File 2007_mid_term_review_learning_from_the_tsunami.pdf saved to disk.
File 2007_risk_disaster_and_management_in_central_america_south_america_and_mexico_concepts_approaches_activities_and_institutional_and_organizational_actors.zip saved to disk.
File 2007_evaluation_of_the_red_cross_movement_livelihoods_programme_for_tsunami_rrecovery_in_sri_lanka.zip saved to disk.
File 2007_risk_disaster_and_management_in_central_america_south_america_and_mexico_concepts_approaches_activities_and

C:\Users\ericf\AppData\Local\Temp\ipykernel_18032\3559517825.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['download_status'] = df2.apply(lambda x: download_docs(x['document_url'], x['doc_filename']), axis=1)


In [27]:
df.to_csv("..\\corpora\\ifrc_evaluations\\document_inventory.csv")

# END

In [ ]:
df_sample['Date'] = pd.to_datetime(df_sample['Date'], dayfirst=True)
df_sample['year'] = df_sample['Date'].dt.year
df_sample['doc_extension'] = df_sample['document_url'].apply(lambda x: x[-3:])
df_sample['doc_filename'] =  df_sample['year'].astype(str) +'_' + df_sample['Title'].apply(lambda x: re.sub(r'\W+', '_', x).lower()) + '.' + df_sample['doc_extension']
df_sample

In [ ]:
url ='https://www.ifrc.org/media/13756'
    
    
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')


results = soup.find_all('a', {"class": "col-3-md download-links__download-link"})
print(results)


In [ ]:
df_sample['doc_filename'] = df_sample['Date'].astype(str)[-4:] + '_' + df_sample['Title'].apply(lambda x: re.sub(r'\W+', '_', x).lower()) + '.' + df_sample['doc_extension']
df_sample

In [ ]:
df_sample['Date'] = pd.to_datetime(df_sample['Date'], dayfirst=True)
df_sample['year'] = df_sample['Date'].dt.year
df_sample